<a href="https://colab.research.google.com/github/rg-smith/remote_sensing_course/blob/main/labs/lab7/Lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 7

**This lab has a similar format to Lab 6, but we will view SAR imagery instead of optical imagery. You will use python (and a module called Google Earth Engine) to view the satellite imagery, evaluate properties that are being imaged, and perform calculations. The lab is divided into four parts:**

**Part 1: Interpreting SAR data over Kansas**

**Part 2: Interpreting SAR data over the Ozarks**

**Part 3: Apply what you've learned to your term project**

**Part 4: Working with InSAR data**

The majority of the code here does not need to be modified, only run. The portions of code that do need to be modified will be shown clearly.

Before starting the sections, we will import the modules we need, initialize Google Earth Engine, and define some functions.

The first block of code only needs to be run once, the first time you open your session (if you close out of the session and open again, you will need to run this again). Follow the prompts to initialize earth engine. You will be taken to a link where you need to give permission to link your google account with Google Earth Engine, then copy and paste some text below the code block.

In [ ]:
import ee
import folium
import numpy as np
import branca.colormap as cm
import gdal

ee.Authenticate()
ee.Initialize()

!git clone https://github.com/rg-smith/remote_sensing_course.git

This block of code also only needs to be run once. It is defining a bunch of functions that you will use in this exercise. It is not important for you to understand the code here for this exercise.

In [ ]:
# functions needed for this lab (and some other useful ones that you can use if you're interested)

# to add a layer to our map:
def add_ee_layer(self, ee_object, name, geometry):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):  
            range = ee.Image(ee_object).reduceRegion(ee.Reducer.percentile([1, 99]),geometry=geometry,scale=300)
            vals = range.getInfo()
            min=list(vals.items())[0][1]
            max=list(vals.items())[1][1]
            vis = {'min': min, 'max': max, 'palette': ['0000FF', 'FFFFFF','FF0000']}

            map_id_dict = ee.Image(ee_object).getMapId(vis)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
            colormap = cm.LinearColormap(vmin=min,vmax=max,colors=['blue', 'white','red']).to_step(n=10)
            colormap.caption=name
            self.add_child(colormap)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except Exception as e: 
        print("Could not display {}".format(name))
        print(e)

# to convert a google earth engine image to a python array
def to_array(img,aoi):
  band_arrs = img.sampleRectangle(region=aoi)

  band_names=band_arrs.bandNames().getInfo()

  for kk in range(len(band_names)):
    if kk==0:
      dat1=np.array(band_arrs.get(band_names[kk]).getInfo())
      dat_full=np.zeros((dat1.shape[0],dat1.shape[1],len(band_names)))
      dat_full[:,:,kk]=dat1
    else:
      dat=np.array(band_arrs.get(band_names[kk]).getInfo())
      dat_full[:,:,kk]=dat
  return(dat_full)

def loadraster(rastername):
    ds=gdal.Open(rastername)
    band=ds.GetRasterBand(1)
    data=band.ReadAsArray()
    # gtr=ds.GetGeoTransform()
    return data

# to calculate an index
def getIndex(image,b1,b2):
  return image.normalizedDifference([b1, b2])

# to calculate a ratio
def getRatio(image1,image2):
  ratio=image1.divide(image2)
  return ratio

# to create a color map from a specific image
def getVisparams(image,aoi):
  range = image.reduceRegion(ee.Reducer.percentile([1, 99]),aoi,300)
  vals = range.getInfo()
  min=list(vals.items())[0][1]
  max=list(vals.items())[1][1]
  visParams = {'min': min, 'max': max, 'palette': ['0000FF', 'FFFFFF','FF0000']}
  return(visParams)

# to get the link to download an earth engine image
def getLink(image,aoi):
  link = image.getDownloadURL({
    'scale': 30,
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    'region': aoi})
  print(link)

# create an earth engine geometry polygon
def addGeometry(min_lon,max_lon,min_lat,max_lat):
  import ee
  geom = ee.Geometry.Polygon(
      [[[min_lon, max_lat],
        [min_lon, min_lat],
        [max_lon, min_lat],
        [max_lon, max_lat]]])
  return(geom)

# load landsat 8 data
def get_l8_image(date1,date2,geometry):
    import ee
    l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT')
    l8_img = l8.filterDate(date1,date2).filterBounds(geometry).first().clip(geometry)
    return(l8_img)

# load landsat 8 data and average all images within time window
def get_l8_image_mean(date1,date2,geometry):
    import ee
    l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT')
    l8_img = l8.filterDate(date1,date2).reduce(ee.Reducer.mean()).clip(geometry)
    print('These are the available bands: ')
    print(l8_img.bandNames().getInfo())
    return(l8_img)

# load sentinel 2 data
def get_s2_image(date1,date2,geometry):
    import ee
    s2 = ee.ImageCollection('COPERNICUS/S2')
    s2_img = s2.filterDate(date1,date2).filterBounds(geometry).first().clip(geometry)
    return(s2_img)

def get_s1_image(date1,date2,geometry,direction):
  if direction == 'ASCENDING' or direction == 'DESCENDING':
    s1 = ee.ImageCollection('COPERNICUS/S1_GRD')
    s1_img = s1.filterDate(date1,date2).filterBounds(geometry).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).select('VV').filter(ee.Filter.eq('orbitProperties_pass', direction)).first()
    return(s1_img)
  else:
    print('inputs are: date1, date2, geometry, direction. Direction should be ASCENDING or DESCENDING')

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Part 1: Interpreting SAR data over Kansas

In this section we will analyze a SAR image over southwest Kansas. This region has extensive agricultural activity, and a few cities. Think about the principles we discussed in class that cause high or low SAR amplitude:

- Dielectric permittivity (i.e. water content)
- Surface roughness
- Geometry

First, we will define the study area and time period and load the Sentinel-1 data

In [ ]:
# create a bounding box that defines the study area
geom_kansas = addGeometry(-100.8, -99.95,37.5,37.85) # min long, max long, min lat, max lat

# define dates of interest (inclusive).
start = '2017-04-01'
end = '2017-05-01'

# pull Sentinel-1 data over the time period and region previously defined. This pulls the first image found in the date range defined
s1_img_kansas_asc = get_s1_image(start,end,geom_kansas,'ASCENDING')

Now, we will view the map. Scroll to the top so you can check/uncheck layers. **Identify five features, some with high amplitude and some with low amplitude. For each feature, take a screenshot, and circle the feature. Make a table that describes whether each feature likely has high/low dielectric permittivity, high/low surface roughness, and whether it likely has a slope/aspect such that it is facing the SAR antenna, facing away from the antenna, or is flat.**

Try to get a variety of features that have different properties for these categories. 

In [ ]:
tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
attr=('Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community')

my_map = folium.Map(location=[37.7, -100], zoom_start=9,tiles=tiles, attr=attr)

# Add the land cover to the map object.
my_map.add_ee_layer(s1_img_kansas_asc,'SAR mean amplitude, ascending track',geom_kansas)
my_map.add_ee_layer(geom_kansas,'bounding box',geom_kansas)
#folium.GeoJson(data=irrigated_corn['geometry']).add_to(my_map)

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

Now, we will download the SAR amplitude, clipped by the bounding box. Load this into ArcMap. Load the 'CornIrr2017.shp' shapefile into ArcMap as well. This is a shapefile of irrigated corn fields in 2017. Do the irrigated fields have higher or lower amplitude than other fields? Why?

In [ ]:
s1_img_small=s1_img_kansas_asc.clip(geom_kansas)
print('SAR amplitude:')
getLink(s1_img_small,geom_kansas)

SAR amplitude:
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/84833f16f566d7948f12837d57ab0302-8ac3d4ce480c60d272453f0d1ff0bf2d:getPixels


# Part 2: Interpreting SAR data over Missouri

For this section, we will repeat Part 1, but over the Ozarks in Missouri. Notice how the features that stand out have some similarities and some differences with the optical data you viewed in the previous lab.

In [ ]:
# create a bounding box that defines the study area. This will cover the Lake of the Ozarks region
geom_ozarks = addGeometry(-92.62,-93.42,38.04,38.40) # min long, max long, min lat, max lat

# define dates of interest (inclusive).
start = '2017-01-01'
end = '2017-04-01'

# pull landsat image over the study area and dates defined
s1_img_ozarks_asc = get_s1_image(start,end,geom_ozarks,'ASCENDING')

Now, you will display the map. **Notice that because the algorithm pulls the first image that intersects the bounding box, in this case it is only covering a small portion of the box. That's OK for this exercise, we just want to view some SAR imagery. **

**Add to your existing table a water feature from the Ozarks SAR image below. Include a screenshot, and all the properties listed in part 1.**

Try to get a variety of features that have different properties for these categories.

In [ ]:
tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
attr=('Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community')

my_map = folium.Map(location=[38.1, -92.92], zoom_start=9,tiles=tiles, attr=attr)

# Add the land cover to the map object.
my_map.add_ee_layer(s1_img_ozarks_asc,'SAR mean amplitude, ascending track',geom_ozarks)
my_map.add_ee_layer(geom_ozarks,'bounding box',geom_ozarks)

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

# Part 3: Apply to your term project

Like last lab, you will now apply what you've learned to your study area. View SAR imagery over your study area, and use it to make some interpretation. It may be relevant to your term project, or may not be. But once again, pick 5 features and identify their three properties that relate to SAR amplitude.

To transfer this to your study area, you will first replace the coordinates (min lon, max lon, etc.) with coordinates of the minimum and maximum longitude and latitude of your study area. (Hint: don't make it too big, or google earth engine will get bogged down. A range of 1 degree or so is usually fine). Then replace the 'yyyy-mm-dd' for start and end to reflect dates that you are interested in. This is Landsat8, so be sure to pick dates when you know Landsat 8 was in orbit. The algorithm will pick the first image it can find, so the start date is the most important.

In [ ]:
# create a bounding box that defines the study area. This will cover the Lake of the Ozarks region
geom_studyarea = addGeometry(minlon,maxlon,minlat,maxlat) # min long, max long, min lat, max lat

# define dates of interest (inclusive).
start = 'yyyy-mm-dd'
end = 'yyyy-mm-dd'

s1_img_studyarea = get_s1_image(start,end,geom_studyarea,'ASCENDING')

View the map. You'll have to replace lat and lon with the coordinates you want the map centered on. Include a screenshot of your map with the index raster in your report.

In [ ]:
my_map = folium.Map(location=[lat, lon], zoom_start=11)

# Add the land cover to the map object.
my_map.add_ee_layer(s1_img_studyarea,'SAR amplitude, ascending track')
my_map.add_ee_layer(geom_studyarea,'bounding box')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

You can download the index if you'd like by running this code:
(if it fails, it will be because the file is too big. You can reduce the file by making your bouding box smaller).

In [ ]:
s1_download=s1_img_studyarea.clip(geom_studyarea)
print('raster download:')
getLink(s1_download,geom_studyarea)

# Part 4: Working with InSAR data

In [ ]:
import gdal
import matplotlib.pyplot as plt

So far we've been looking at amplitude from SAR data. Now we will look at phase. The phase of a single image will look random:

In [ ]:
phase_single=np.random.uniform(-np.pi,np.pi,(553,737))
plt.figure(figsize=(7,5));plt.imshow(phase_single);plt.colorbar()

But the change in phase from one satellite pass to another (or from the same satellite pass, if the satellite is viewing from multiple angles) can tell us something about the absolute elevation, and the change in elevation (if the acquisitions happen at different times). The change in phase from one acquisition to another is called interferometric synthetic aperture radar (InSAR).

The image below is an interferogram, or the change in phase from one satellite pass to another. Notice that the fringe patterns look a bit like contour lines. This is because they are related to elevation. As elevation changes, the distance the wave travels from the satellite to the ground and back also changes, resulting in a change in phase. This change in phase resets each time the wave completes a cycle.

In [ ]:
insar_wrapped = loadraster('remote_sensing_course/labs/lab7/insar_wrapped.tif')
plt.figure(figsize=(12,10))
plt.imshow(insar_wrapped);plt.colorbar()

There is a processing tool called 'unwrapping' that finds the fringe patterns to determine where the wave has cycled, and increases their phase so that the phase change is continuous, thus removing the fringes. This is shown below.

In [ ]:
insar_unwrapped = loadraster('remote_sensing_course/labs/lab7/insar_unwrapped.tif')
plt.figure(figsize=(12,10))
plt.imshow(insar_unwrapped);plt.colorbar()

This image is much more meaningful, and it's clear it's related to elevation. But we need to do some work to figure out what the elevation is. Use the formula from class to determine how to transform the change in phase, in radians, to elevation, in m. The factor that converts from change in phase, in radians, to elevation, h, in m, is given in Lecture 6 and in Equation 5.2 in  [Lu et al. (2021)](https://www.smu.edu/-/media/Site/Dedman/Academics/Departments/EarthSciences/PDF/Lu/098_Lu_etal_InSAR_DEM_2012.pdf?la=en) (assume deformation is 0). Parameters you will need are given in the code below.

In [ ]:
theta = 0.61
H=693000
B=300
lambda_=0.236

# make a line here to calculate the conversion factor from phase to elevation (change in elevation, m divided by change in phase, radians)
conversion_factor=

# make a line here to multiply the insar_unwrapped file by the conversion factor, transforming it to elevation, in m


Once you've done the transformation, plot the image that you created. It should have values that are more representative of true elevation. Use the same code as above (plt.figure, plt.imshow) to plot your elevation image.

**Include the code you used to convert it to elevation, and a screenshot of your elevation plot.**

In [ ]:
# fill this in with code to plot the image you created (follow the pattern for plotting figures from previous examples)
# plt.figure(figsize=(12,10))
# plt.imshow(<your varible name>);plt.colorbar()


As the Lu et al. chapter above indicates, change in phase is sensitive to the absolute elevation, which you just calculated, and to the change in elevation. If the elevation is known, its contribution to the change in phase can be subtracted from the interferogram. Then, the remaining change in phase is only due to ground deformation (if we assume negligible changes in atmospheric water vapor and orbital errors, which we will for this exercise).

We will load the known elevation for this area:

In [ ]:
elev = loadraster('remote_sensing_course/labs/lab7/dem.tif')
plt.figure(figsize=(12,10))
plt.imshow(elev);plt.colorbar()

Now, convert this raster from elevation, in m to change in phase, in radians. You will use the same conversion factor you already calculated, but this time divide instead of multiply.

In [ ]:
# fill this in with code to calculate the change in phase from the elevation raster, 'elev', and plot it


Now, make a 'flattened' raster that subtracts the change in phase that you calculated from the 'insar_unwrapped' raster. Plot this image. This now shows the deformation that occurred between satellite passes, in radians! This can be converted to m using: phase x lambda / (pi x 4) 

**Include the image showing deformation, and the code you used to calculate and subtract the elevation phase from the original interferogram in your lab report.**

In [ ]:
# fill this in with code to subtract your phase calculated from the elevation from 'insar_unwrapped'

# fill this in with code to plot the image you just created
